# gestion des secrets

[source](https://savelev.medium.com/best-practice-how-to-store-secrets-and-settings-in-python-project-e3ee45b3094c)

In [2]:
import os
import json
def read_secrets() -> dict:
    filename = os.path.join('secrets.json')
    try:
        with open(filename, mode='r') as f:
            return json.loads(f.read())
    except FileNotFoundError:
        return {}
secrets = read_secrets()

# test accès stripe

In [3]:
import stripe
stripe.api_key = secrets['stripe.api_key_test']

# list customers
customers = stripe.Customer.list()

# print the first customer's email
print(customers.data[0].email)

# retrieve specific Customer
# customer = stripe.Customer.retrieve("cus_123456789")

# print that customer's email
# print(customer.email)

cha.le@hotmail.fr


# exeuction rapport

In [4]:
import datetime
import time

## gestion date début/fin rapport

[lien vers doc de stripe](https://stripe.com/docs/api/reporting/report_run/object)

In [37]:
def convert_to_timestamp(date):
	return int(time.mktime(datetime.datetime.strptime(date, "%d/%m/%Y").timetuple()))

start_date = "01/10/2022"
end_date = "29/10/2022"
start_date_timestamp = convert_to_timestamp(start_date)
end_date_timestamp = convert_to_timestamp(end_date)

In [21]:
import stripe
stripe.api_key = secrets['stripe.api_key_test']

report = stripe.reporting.ReportRun.create(
  report_type="payout_reconciliation.itemized.5",
  parameters={
    "interval_start": start_date_timestamp,
    "interval_end": end_date_timestamp,
    "columns": ['automatic_payout_id'
  ]
  },
  
)

report['id']

<ReportRun reporting.report_run id=frr_1Lye0nJeZLXFAOnZPi3JYTh8 at 0x7f03b6417f60> JSON: {
  "created": 1667145925,
  "error": null,
  "id": "frr_1Lye0nJeZLXFAOnZPi3JYTh8",
  "livemode": false,
  "object": "reporting.report_run",
  "parameters": {
    "interval_end": 1667080800,
    "interval_start": 1664575200
  },
  "report_type": "payout_reconciliation.itemized.1",
  "result": null,
  "status": "pending",
  "succeeded_at": null
}

In [29]:
report['status']


'succeeded'

In [59]:

import stripe
import requests
import pandas as pd
import io
stripe.api_key = secrets['stripe.api_key_test']

PAUSE = 10
start_date = "01/12/2021"
end_date = "20/12/2021"
start_date_timestamp = convert_to_timestamp(start_date)
end_date_timestamp = convert_to_timestamp(end_date)

report = stripe.reporting.ReportRun.create(
  report_type="payout_reconciliation.itemized.1",
  parameters={
    "interval_start": start_date_timestamp,
    "interval_end": end_date_timestamp,
  },
)


timeout = time.time() + 60*5   # 1 minutes from now
i=0

id = report['id']

while time.time() < timeout:
    report = stripe.reporting.ReportRun.retrieve(id)
    if report['status'] == 'succeeded':
        print(f"Execution {i+1} +{PAUSE*i}sec : {report['status']}")
        r = requests.get(report['result']['url'], auth=(stripe.api_key,""))
        urlData = r.content
        rawData = pd.read_csv(io.StringIO(urlData.decode('utf-8')))
        break
    else :
        print(f"Execution {i+1} +{PAUSE*i}sec : {report['status']}")
    i +=1
    time.sleep(PAUSE)

Execution 1 +0sec : pending
Execution 2 +10sec : pending
Execution 3 +20sec : pending
Execution 4 +30sec : pending
Execution 5 +40sec : pending
Execution 6 +50sec : pending
Execution 7 +60sec : pending
Execution 8 +70sec : pending
Execution 9 +80sec : pending
Execution 10 +90sec : pending
Execution 11 +100sec : pending
Execution 12 +110sec : pending
Execution 13 +120sec : pending
Execution 14 +130sec : pending
Execution 15 +140sec : pending
Execution 16 +150sec : pending
Execution 17 +160sec : pending
Execution 18 +170sec : pending
Execution 19 +180sec : pending
Execution 20 +190sec : pending
Execution 21 +200sec : pending
Execution 22 +210sec : pending
Execution 23 +220sec : pending
Execution 24 +230sec : pending
Execution 25 +240sec : pending
Execution 26 +250sec : pending
Execution 27 +260sec : pending
Execution 28 +270sec : pending
Execution 29 +280sec : pending
Execution 30 +290sec : pending


In [63]:
report = stripe.reporting.ReportRun.retrieve(id)
r = requests.get(report['result']['url'], auth=(stripe.api_key,""))
urlData = r.content
rawData = pd.read_csv(io.StringIO(urlData.decode('utf-8')))

In [65]:
rawData.groupby('reporting_category').sum()

/tmp/ipykernel_515/3283019274.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  rawData.groupby('reporting_category').sum()


,gross,fee,net,customer_facing_amount
reporting_category,,,,
charge,585.0,20.2,564.8,585.0
payout,-360.3,0.0,-360.3,0.0
refund,-204.5,0.0,-204.5,-204.5


In [64]:
rawData

,balance_transaction_id,created_utc,available_on_utc,currency,gross,fee,net,reporting_category,source_id,description,customer_facing_amount,customer_facing_currency,automatic_payout_id,automatic_payout_effective_at,charge_id
0,txn_3JzLxOJeZLXFAOnZ0bp9gwol,2021-11-24 13:56:19,2021-12-01 00:00:00,eur,20.50,0.84,19.66,charge,ch_3JzLxOJeZLXFAOnZ0fLo84hd,APE Dracy-le-Fort - Commande 48,20.5,eur,po_1Jzt0CJeZLXFAOnZu11ydCZv,2021-12-01 00:00:00,ch_3JzLxOJeZLXFAOnZ0fLo84hd
1,txn_3JzLxOJeZLXFAOnZ0vJI077H,2021-11-24 13:59:52,2021-12-01 00:00:00,eur,-20.50,0.00,-20.50,refund,re_3JzLxOJeZLXFAOnZ0ITGt7pW,REFUND FOR CHARGE (APE Dracy-le-Fort - Command...,-20.5,eur,po_1Jzt0CJeZLXFAOnZu11ydCZv,2021-12-01 00:00:00,ch_3JzLxOJeZLXFAOnZ0fLo84hd
2,txn_3JzM80JeZLXFAOnZ1jEJwPnk,2021-11-24 14:07:17,2021-12-01 00:00:00,eur,68.00,2.22,65.78,charge,ch_3JzM80JeZLXFAOnZ1wntO3rM,APE Dracy-le-Fort - Commande 50,68.0,eur,po_1Jzt0CJeZLXFAOnZu11ydCZv,2021-12-01 00:00:00,ch_3JzM80JeZLXFAOnZ1wntO3rM
3,txn_3JzMzNJeZLXFAOnZ1p3Zl1AJ,2021-11-24 15:02:26,2021-12-01 00:00:00,eur,109.50,3.43,106.07,charge,ch_3JzMzNJeZLXFAOnZ1EajBVAC,APE Dracy-le-Fort - Commande 54,109.5,eur,po_1Jzt0CJeZLXFAOnZu11ydCZv,2021-12-01 00:00:00,ch_3JzMzNJeZLXFAOnZ1EajBVAC
4,txn_3JzMzNJeZLXFAOnZ1j9QyRD8,2021-11-24 15:06:12,2021-12-01 00:00:00,eur,-109.50,0.00,-109.50,refund,re_3JzMzNJeZLXFAOnZ15ukW1O4,REFUND FOR CHARGE (APE Dracy-le-Fort - Command...,-109.5,eur,po_1Jzt0CJeZLXFAOnZu11ydCZv,2021-12-01 00:00:00,ch_3JzMzNJeZLXFAOnZ1EajBVAC
5,txn_3JzOI5JeZLXFAOnZ19FQbcg1,2021-11-24 16:25:50,2021-12-01 00:00:00,eur,15.50,0.70,14.80,charge,ch_3JzOI5JeZLXFAOnZ1zxuY5YU,APE Dracy-le-Fort - Commande 56,15.5,eur,po_1Jzt0CJeZLXFAOnZu11ydCZv,2021-12-01 00:00:00,ch_3JzOI5JeZLXFAOnZ1zxuY5YU
6,txn_3JzS0PJeZLXFAOnZ02BlTrn3,2021-11-24 20:23:50,2021-12-01 00:00:00,eur,63.00,2.08,60.92,charge,ch_3JzS0PJeZLXFAOnZ0SyJe6T1,APE Dracy-le-Fort - Commande 62,63.0,eur,po_1Jzt0CJeZLXFAOnZu11ydCZv,2021-12-01 00:00:00,ch_3JzS0PJeZLXFAOnZ0SyJe6T1
7,txn_3JzS6HJeZLXFAOnZ1txE9Glc,2021-11-24 20:29:54,2021-12-01 00:00:00,eur,20.50,0.84,19.66,charge,ch_3JzS6HJeZLXFAOnZ1rlYPmKH,APE Dracy-le-Fort - Commande 63,20.5,eur,po_1Jzt0CJeZLXFAOnZu11ydCZv,2021-12-01 00:00:00,ch_3JzS6HJeZLXFAOnZ1rlYPmKH
8,txn_3JzSF7JeZLXFAOnZ1HjAuyK5,2021-11-24 20:39:02,2021-12-01 00:00:00,eur,27.00,1.03,25.97,charge,ch_3JzSF7JeZLXFAOnZ1HwA4TfQ,APE Dracy-le-Fort - Commande 64,27.0,eur,po_1Jzt0CJeZLXFAOnZu11ydCZv,2021-12-01 00:00:00,ch_3JzSF7JeZLXFAOnZ1HwA4TfQ
9,txn_3JzSHLJeZLXFAOnZ14yFHWJt,2021-11-24 20:41:20,2021-12-01 00:00:00,eur,74.50,2.41,72.09,charge,ch_3JzSHLJeZLXFAOnZ10N4Ez5W,APE Dracy-le-Fort - Commande 65,74.5,eur,po_1Jzt0CJeZLXFAOnZu11ydCZv,2021-12-01 00:00:00,ch_3JzSHLJeZLXFAOnZ10N4Ez5W


In [58]:
r.content

b'"balance_transaction_id","created_utc","available_on_utc","currency","gross","fee","net","reporting_category","source_id","description","customer_facing_amount","customer_facing_currency","automatic_payout_id","automatic_payout_effective_at","charge_id"\n'

In [44]:
import pandas as pd
import io

urlData = r.content
rawData = pd.read_csv(io.StringIO(urlData.decode('utf-8')))

In [45]:
rawData

,balance_transaction_id,created_utc,available_on_utc,currency,gross,fee,net,reporting_category,source_id,description,customer_facing_amount,customer_facing_currency,automatic_payout_id,automatic_payout_effective_at,charge_id
